## Automação Web

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [20]:
# Importar/Visualizar a Base de Dados
produtos_df = pd.read_excel('Buscas.xlsx')
display(produtos_df)

print('\n\n')
produtos_df.info()

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch usado,3000,4000
1,rtx 3060,zota galax usado,4000,4500
2,playstation 5,controle fone usado,3000,5500





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Nome            3 non-null      object
 1   Termos banidos  3 non-null      object
 2   Preço mínimo    3 non-null      int64 
 3   Preço máximo    3 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 224.0+ bytes


In [25]:
# Criar um Navegador
driver = webdriver.Chrome()

## Definição das Funções de Busca no Google e no Buscapé

In [22]:
# Busca no Google
def busca_google_shopping(driver, produto, termos_banidos, preco_minimo, preco_maximo):
    
    # Entrar no google
    driver.get('https://www.google.com/')
    
    # Tratar os Valores que Vieram da Tabela
    produto = produto.lower()
    lista_termos_produto = produto.split(' ')
    
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
        
    # Pesquisar o Nome do Produto no Google
    while len(driver.find_elements(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')) == 0:
        time.sleep(1)
    
    time.sleep(1)
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    
    # Clicar na Aba Shopping
    while len(driver.find_elements(By.XPATH, '//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input')) == 0:
        time.sleep(1)
    
    time.sleep(1)
    elementos = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    
    for item in elementos:
        if 'Shopping' in item.text:
            item.click()
            break

    # Lista de Resultados da Busca no Google Shopping
    while len(driver.find_elements(By.XPATH, '//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input')) == 0:
        time.sleep(1)

    time.sleep(1)
    lista_resultados = driver.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    
    # Para Cada Resultado, Verificar se o Resultado Corresponde a Todas as Condições
    lista_ofertas = []
    
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
        nome = nome.lower()

        # Verificação do Nome - Se no Nome Tem Algum Termo Banido
        tem_termos_banidos = False
        
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True
        
        # Verificação do Nome - Se no Nome Tem Todos os Termos do Nome do Produto
        tem_todos_termos_produto = True
        
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False
        
        # Verificar se o Nome Atende as Condições
        if not tem_termos_banidos and tem_todos_termos_produto:            # ou -> if tem_termos_banidos == False and tem_todos_termos_produto == True:
            try:
                # Tratando o Preço do Produto
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                
                # Verificando se o Preco Está Dentro do Mínimo e Máximo
                if preco_minimo <= preco <= preco_maximo:
                    elemento_link = resultado.find_element(By.CLASS_NAME, 'aULzUe')
                    # ('..') -> Busca o Elemento de Cima, Nesse Caso, o Elemento Pai
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
            except:
                continue

    return lista_ofertas

In [23]:
# Busca no Buscapé
def busca_buscape(driver, produto, termos_banidos, preco_minimo, preco_maximo):
    
    # Entrar no Buscapé
    driver.get('https://www.buscape.com.br/')
    
    # Tratar os Valores que Vieram da Tabela
    produto = produto.lower()
    lista_termos_produto = produto.split(' ')
    
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    
    # Pesquisar o Nome do Produto no Buscapé
    while len(driver.find_elements(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[1]/input')) == 0:
        time.sleep(1)

    time.sleep(1)
    driver.find_element(By.CLASS_NAME, 'search-bar__text-box').send_keys(produto)
    driver.find_element(By.CLASS_NAME, 'search-bar__text-box').send_keys(Keys.ENTER)
    
    # Lista de Resultados da Busca no Buscapé
    while len(driver.find_elements(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[4]/div/a/span[2]')) == 0:
        time.sleep(1)
    
    time.sleep(5)
    lista_resultados = driver.find_elements(By.CLASS_NAME, 'Cell_Content__1630r')
    
    # Para Cada Resultado, Verificar se o Resultado Corresponde a Todas as Condições
    lista_ofertas = []
    
    for resultado in lista_resultados:
        nome = resultado.get_attribute('title')
        nome = nome.lower()

        # Verificação do Nome - Se no Nome Tem Algum Termo Banido
        tem_termos_banidos = False

        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True  

        # Verificação do Nome - Se no Nome Tem Todos os Termos do Nome do Produto
        tem_todos_termos_produto = True

        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False            

        # Verificar se o Nome Atende as Condições
        if not tem_termos_banidos and tem_todos_termos_produto:
            try:
                # Tratando o Preço do Produto
                preco = resultado.find_element(By.CLASS_NAME, 'CellPrice_MainValue__3s0iP').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)

                # Verificando se o Preço Está Dentro do Mínimo e Máximo
                if preco_minimo <= preco <= preco_maximo:
                    link = resultado.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))

            except:
                continue
                        
    return lista_ofertas

## Contrução da Nossa Lista de Ofertas Encontradas

In [26]:
tabela_ofertas = pd.DataFrame()

# Dados de Entrada para as Funções Criadas 
for linha in produtos_df.index:
    produto = produtos_df.loc[linha, "Nome"]
    termos_banidos = produtos_df.loc[linha, "Termos banidos"]
    preco_minimo = produtos_df.loc[linha, "Preço mínimo"]
    preco_maximo = produtos_df.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(driver, produto, termos_banidos, preco_minimo, preco_maximo)
    
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns = ['Produto', 'Preço (R$)', 'Link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(driver, produto, termos_banidos, preco_minimo, preco_maximo)
    
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns = ['Produto', 'Preço (R$)', 'Link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    
    else:
        tabela_buscape = None

tabela_ofertas = tabela_ofertas.reset_index(drop = True)
display(tabela_ofertas)

,Produto,Preço (R$),Link
0,smartphone apple iphone 12 64gb tela 61 camera...,3784.45,https://www.google.com/url?url=https://www.bar...
1,iphone 12 64 gb | celular iphonne,3650.00,https://www.google.com/url?url=https://www.enj...
2,iphone 11 tela 6.1'' / 12mp / ios 13 vermelho ...,3790.00,https://www.google.com/url?url=https://www.wil...
3,iphone 11 tela 6.1'' / 12mp / ios 13 branco / ...,3790.00,https://www.google.com/url?url=https://www.wil...
4,smartphone apple iphone xr 64gb ios 12.0 mp,3479.00,https://www.buscape.com.br/celular/smartphone-...
5,smartphone apple iphone xr vermelho 64gb ios 1...,3998.00,https://www.buscape.com.br/celular/smartphone-...
6,"iphone xs space gray, com tela de 5,8, 4g, 64 ...",3347.56,https://www.buscape.com.br/lead?oid=714484000&...
7,"iphone xs space gray, com tela de 5,8, 4g, 64 ...",3347.56,https://www.buscape.com.br/lead?oid=714417477&...
8,placa de vídeo gigabyte geforce rtx 3060 eagle...,4199.90,https://www.google.com/url?url=https://www.pic...
9,placa de vídeo palit geforce rtx 3060 12gb - g...,4470.57,https://www.google.com/url?url=https://www.gki...


## Exportar a Base de Ofertas para Excel

In [27]:
# Exportar Para o Excel
tabela_ofertas.to_excel("Ofertas.xlsx", index = False)

## Enviando o E-mail

In [83]:
# Enviar Por E-mail o Resultado da Tabela
import win32com.client as win32

# Verificando se Existe Alguma Oferta Dentro da Tabela de Ofertas
if len(tabela_ofertas.index) > 0:
    outlook = win32.Dispatch('outlook.application')
    emissor = outlook.session.Accounts['gabriel_rcn@hotmail.com']

    message = outlook.CreateItem(0)
    
    message.To = 'email_destinatario@gmail.com'
    
    message.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    
    message.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index = False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    
    message._oleobj_.Invoke(*(64209,0,8,0,emissor))

    message.Save()
    
    mail.Send()

driver.quit()